<a href="https://colab.research.google.com/github/ikoojos/Algorithm-Debt-Research/blob/master/Random_Forest_Hash_Vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
from itertools import product
from sklearn.ensemble import RandomForestClassifier
import importlib

drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/AD Final Experiments')

from preprocessing import preprocess_data
from splitting import split_data
from utils import *
from evaluate_model import evaluate_best_model
from rf_tuning import hyperparameter_tuning_rf


for module in ['preprocessing', 'splitting', 'utils', 'evaluate_model', 'rf_tuning']:
    importlib.reload(sys.modules[module])

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2]
}

vectorizer = HashingVectorizer()
best_model, best_params, best_score = hyperparameter_tuning_rf(X_train_final, y_train_final, X_val, y_val, param_grid, vectorizer)
evaluate_best_model(best_model, best_params, best_score, X_test, y_test)

Best parameters found: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1}
Validation set accuracy: 0.8790996784565916

Test Confusion Matrix:
[[  61    0    1   95    0    1    0   42]
 [   0   32    0   41    0    4    0   12]
 [   0    0   39   71    0    1    1   23]
 [   3    2    4 1977    0   33    3  184]
 [   0    0    0   13    7    0    0    3]
 [   0    0    2  140    0  209    2   34]
 [   0    0    1   54    0    5   61   22]
 [   3    1    1  128    1    0    0 4458]]

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.91      0.30      0.46       200
         COMPATIBILITY       0.91      0.36      0.52        89
                DEFECT       0.81      0.29      0.43       135
                DESIGN       0.78      0.90      0.84      2206
         DOCUMENTATION       0.88      0.30      0.45        23
        IMPLEMENTATION       0.83      0.54      0.65       38